Reference: https://mp.weixin.qq.com/s/gkTYUvICBpmGtqj9mL_w0A

AIGC即AI Generated Content，是指利用人工智能技术来生成内容，AIGC也被认为是继UGC、PGC之后的新型内容生产方式，AI绘画、AI写作等都属于AIGC的分支。

本次比赛的任务是创建一个模型，该模型可以反转生成给定图像的扩散过程（即通过图片生成原始prompt）。

输入：16,000张SD生成的图片；

标签：prompt文本编码之后的向量；

https://www.kaggle.com/competitions/stable-diffusion-image-to-prompts

使用预测和实际提示嵌入向量之间的余弦相似度评估，对于测试集中的每个图像，您必须预测用于生成图像的prompt转换为 384 长度的嵌入向量。

In [ ]:
# 核心评分代码如下，对文本通过all-MiniLM-L6-v2进行编码，然后转换维度进行提交

# https://www.kaggle.com/code/inversion/stable-diffusion-sample-submission/

# 读取prompt文本
sample_submission = pd.read_csv('sample_submission.csv', index_col='imgId_eId')

# 读取用于编码prompt的模型
st_model = SentenceTransformer('/kaggle/input/sentence-transformers-222/all-MiniLM-L6-v2')

# 编码并转换维度
prompt_embeddings = st_model.encode(prompts['prompt']).flatten()

# 写入提交结果
submission = pd.DataFrame(
                index=imgId_eId,
                data=prompt_embeddings,
                columns=['val']).rename_axis('imgId_eId')
submission.to_csv('submission.csv')

Stable Diffusion介绍:

Stable Diffusion（简称SD）诞生于2022年8月，主要思路来自于CVPR22 上合作发表的潜扩散模型（Latent Diffusion Model）论文。

相比较于DALL-E等大模型，Stable Diffusion对显存压力更小。Stable Diffusion不仅生成的图像质量非常高，运行速度快，并且有资源和内存的要求也较低。

Stable Diffusion根据文本提示作为输入来生成的图像，也可以用它对图像根据文字描述进行修改。


SD Prompt规则:

Prompt顺序：越靠前的Tag权重越大：比如景色Tag在前，人物就会小，相反的人物会变大或半身。

Prompt个数：生成图片的大小会影响Prompt的效果，图片越大需要的Prompt越多，不然Prompt会相互污染。

Prompt权重：在Stable Diffusion 中使用()英文括号可增加括号中Tag在画面中的权重 x1.1，[]可减小Tag权重x0.91。

Emoji Prompt：Prompt支持使用emoji，且表现力较好，可通过添加emoji达到表现效果，如😍形容表情。

更多SD相关的Prompt特征工程，可以参考如下资料：

- https://github.com/Maks-s/sd-akashic
- https://github.com/adieyal/sd-dynamic-prompts
- https://invoke-ai.github.io/InvokeAI/features/PROMPTS/

all-MiniLM-L6-v2在可用训练数据（超过 10 亿个句子对）上进行训练，将句子和段落映射到 384 维密集向量空间，可用于聚类或语义搜索等任务。

更多模型介绍，可以参考如下资料

- https://www.sbert.net/docs/pretrained_models.html
- https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2

赛题是AIGC领域的前沿赛题，但并不是单纯创造图片，而是希望构建模型去生成合理的prompt，进而生成合适的图片。

$$
    图片---->prompt---->图片
$$

赛题使用的Stable Diffusion v2（SD v2）权重是公开的，且可以轻松在本地或者运算生成新的图片。模型权重和地址如下：

- 模型权重：https://huggingface.co/stabilityai/stable-diffusion-2/blob/main/768-v-ema.ckpt
- 使用代码：https://github.com/Stability-AI/stablediffusion/blob/main/scripts/txt2img.py

本次赛题并不提供数据，而是提供了一个模型权重，需要选手通过模型权重来得到的一个逆向的模型。

In [ ]:
# 思路1：Vit-GPT2
# Visual Transformers（Vit）是强大的图像分类模型，
# 而 GPT2 是一种可用于生成文本的语言模型。Vit-GPT2思路是将图片Vit特征解码为文本，用来描述图片内容
# Baseline地址：https://huggingface.co/nlpconnect/vit-gpt2-image-captioning

# 加载Vit-GPT模型
model = VisionEncoderDecoderModel.from_pretrained(model_dir)
feature_extractor = ViTFeatureExtractor.from_pretrained(model_dir)
tokenizer = AutoTokenizer.from_pretrained(model_dir)

# 对图片生成文本，然后对文本进行编码
def predict_step(image_paths):
  images = []
  for image_path in image_paths:
    i_image = Image.open(image_path)
    if i_image.mode != "RGB":
      i_image = i_image.convert(mode="RGB")

    images.append(i_image)
  
  # Vit-GPT，Vit部分
  pixel_values = feature_extractor(images=images, return_tensors="pt").pixel_values
  pixel_values = pixel_values.to(device)
  # Vit-GPT，GPT部分
  output_ids = model.generate(pixel_values, **gen_kwargs)
  # 文本编码部分
  preds = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
  preds = [pred.strip() for pred in preds]
  return preds


In [ ]:
# 思路2：OFA模型
# OFA将多模态及单模态的理解和生成任务统一到一个简单的Seq2Seq生成式框架中，OFA执行预训练并使用任务指令进行微调。
# OFA模型资料如下：
# - 开源地址：https://github.com/OFA-Sys/OFA
# - 模型权重：https://huggingface.co/OFA-Sys
# Baseline地址：https://www.kaggle.com/code/mayukh18/ofa-transformer-lb-0-42644

# 图片数据变换
mean, std = [0.5, 0.5, 0.5], [0.5, 0.5, 0.5]
resolution = 480
patch_resize_transform = transforms.Compose([
        lambda image: image.convert("RGB"),
        transforms.Resize((resolution, resolution), interpolation=Image.BICUBIC),
        transforms.ToTensor(), 
        transforms.Normalize(mean=mean, std=std)
    ])

# 加载OFA模型，输入任务指令
tokenizer = OFATokenizer.from_pretrained(CKPT_DIR)
model = OFAModel.from_pretrained(CKPT_DIR, use_cache=False).cuda()
txt = " what does the image describe?"
inputs = tokenizer([txt], return_tensors="pt").input_ids

In [ ]:
# 思路3：CLIP + Prompt模板
# CLIP从互联网收集的4亿(图像、文本)对的数据集，在预训练之后，用自然语言描述所学的视觉概念，从而使模型能够在zero-shot状态下转移到下游任务。
# Baseline地址：https://www.kaggle.com/code/leonidkulyk/lb-0-45836-blip-clip-clip-interrogator

def interrogate(image: Image) -> str:
    # CLIP预测得到文本
    caption = ci.generate_caption(image)
    
    # 计算图片特征
    image_features = ci.image_to_features(image)
    
    # 计算博物馆prompt
    medium = [ci.mediums.labels[i] for i in cos(image_features, mediums_features_array).topk(1).indices][0]
    # 计算图片风格prompt
    movement = [ci.movements.labels[i] for i in cos(image_features, movements_features_array).topk(1).indices][0]
    # 计算口味/通用prompt
    flaves = ", ".join([ci.flavors.labels[i] for i in cos(image_features, flavors_features_array).topk(3).indices])
    
    # prompt模板
    if caption.startswith(medium):
        prompt = f"{caption}, {movement}, {flaves}"
    else:
        prompt = f"{caption}, {medium}, {movement}, {flaves}"

    return clip_interrogator._truncate_to_fit(prompt, ci.tokenize)